In [65]:
from pathlib import Path
from dataclasses import dataclass
import os
import pandas as pd

os.getcwd()
os.chdir("C:\\Users\somit\Downloads\project_ineuron\insurence_premium_prediction")
os.getcwd()

'C:\\Users\\somit\\Downloads\\project_ineuron\\insurence_premium_prediction'

In [66]:
@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    schema_file_path:Path
    report_file_path: Path
    report_page_file_path: Path
    
    

In [75]:
from insurence_premium.constant import *
from insurence_premium.util.common import read_yaml ,create_directories 

In [1]:
from insurence_premium.constant import SCHEMA_FILE_PATH

ImportError: cannot import name 'SCHEMA_FILE_PATH' from 'insurence_premium.constant' (c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\site-packages\insurencepremium-0.0.1-py3.8.egg\insurence_premium\constant\__init__.py)

In [ ]:
class ConfigurationManager:
    def __init__(
        self ,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config =read_yaml(config_filepath)
        self.params =read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    

    def get_validation_config(self)->DataValidationConfig:
        validation_config =self.config.data_validation

        data_validation_config =DataValidationConfig(
            root_dir=Path(validation_config.root_dir),
            schema_file_path= SCHEMA_FILE_PATH ,
            report_file_path=Path(validation_config.report_file_path),
            report_page_file_path=Path(validation_config.report_page_file_path)
            
        )
        return data_validation_config



In [74]:
con= ConfigurationManager().get_validation_config()

[2023-03-05 01:11:54,219: INFO: common]: yaml file: config\config.yaml loaded successfully
[2023-03-05 01:11:54,225: INFO: common]: yaml file: params.yaml loaded successfully
[2023-03-05 01:11:54,229: INFO: common]: created directory at: artifacts


NameError: name 'SCHEMA_FILE_PATH' is not defined

In [6]:
con.get_validation_config()

DataValidationConfig(root_dir=WindowsPath('artifacts/data_validation'), report_file_path=WindowsPath('artifact/data_validation/report.json'), report_page_file_path=WindowsPath('artifact/data_validation/report.html'))

In [ ]:
class DataValidation:
    def __init__(self ,config:ConfigurationManager):
        self.config =ConfigurationManager()
        self.data_ingestion_config =ConfigurationManager().get_data_ingestion_config()
        self.data_validation_config =ConfigurationManager().get_validation_config()

    def get_train_test_data(self):
        try:
            train_file_path=self.data_ingestion_config.ingested_train_dir
            test_file_path=self.data_ingestion_config.ingested_test_dir
            train_df=pd.read_csv(train_file_path)
            test_df=pd.read_csv(test_file_path)

            return train_df ,test_df
            
        except Exception as e:
            raise e    

    def is_train_test_file_exist(self):
        try:
            is_train_file_exist=False
            is_test_file_exist=False

            train_file_path=self.data_ingestion_config.ingested_train_dir
            test_file_path=self.data_ingestion_config.ingested_test_dir

            is_train_file_exist=os.Path.exists(train_file_path)
            is_test_file_exists=os.Path.exists(test_file_path)

            return is_train_file_exist and is_test_file_exist

            if not (is_train_file_exist and is_train_file_exist):
                raise Exception(f"train or test file missing")
            logger.info(f"train_file: {train_file_path} ,test_file: {test_file_path}") 


        except Exception as e:
            raise e

    def validate_schema(self):
        try:
            train_file_path =self.data_ingestion_config.train_file_path
            
            pd.read_csv(train_file_path)
            
            data_type=True
            if data_type:

                for i in df.columns:
                    df[i].dtypes==schema[i]
                    logger.info(f"data column: {i} is matching with schema data set")
            else:
                logger.info(f"datatypes are not maching column:{i} with schema data")        
        except Exception as e:
            raise e        

    def get_and_save_data_drift_report(self):
        try:
            profile =Profile(sections=[DataDriftProfileSection()])
            train_df,test_df =self.get_train_and_test_df()
            profile.calculate(train_df,test_df)
            report=json.loads(profile.json())
            report_file_path = self.data_validation_config.report_file_path
            report_dir = os.path.dirname(report_file_path)
            os.makedirs(report_dir,exist_ok=True)    

            write_json(report_file_path,report)
            return report
        except Exception as e:
            raise e    

    def save_data_drift_report_page(self):
        try:
            dashboard =Dashboard(tabs=[DataDriftTab()])
            train_df,test_df =self.get_train_and_test_df()
            dashboard.calculate(train_df,test_df)
            report_page_file_path=self.data_validation_config.report_page_file_path
            report_page_dir =os.path.dirname(report_page_file_path)
            os.makedirs(report_page_dir , exist_ok=True)

            dashboard.save(report_page_file_path)
            
        except Exception as e:
            raise e        

    def is_data_drift_found(self):
        try:
            report = self.get_and_save_data_drift_report()
            self.save_data_drift_report_page()
            return True
        except Exception as e:
            raise e              

In [16]:
df=pd.read_csv("C:\\Users\\somit\\Downloads\\project_ineuron\\insurence_premium_prediction\\artifacts\data_ingestion\\train\\insurence.csv",index_col='Unnamed: 0')

In [17]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'expenses'], dtype='object')

In [24]:
df.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
expenses    float64
dtype: object

In [30]:
df.dtypes.index  , df.dtypes.values

(Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'expenses'], dtype='object'),
 array([dtype('int64'), dtype('O'), dtype('float64'), dtype('int64'),
        dtype('O'), dtype('O'), dtype('float64')], dtype=object))

In [31]:
data_type =list(map(lambda x:str(x).replace("dtype('" ,"").replace("')" , "") ,df.dtypes.values))

In [39]:
columns=df.columns

In [37]:
data_type=list(map(lambda x:str(x).replace("dtype('" ,"").replace("')" , "") ,df.dtypes.values))

In [41]:
dict(zip(columns ,data_type))

{'age': 'int64',
 'sex': 'object',
 'bmi': 'float64',
 'children': 'int64',
 'smoker': 'object',
 'region': 'object',
 'expenses': 'float64'}

In [42]:
schema=read_yaml(Path("config/schema.yaml"))

[2023-03-04 22:55:07,237: INFO: common]: yaml file: config\schema.yaml loaded successfully


In [51]:
schema['age']==df['age'].dtypes

True

In [56]:
from insurence_premium import logger

In [52]:
for i in df.columns:
    df[i].dtypes==schema[i]
        

In [59]:
def validate_schema(df):
    data_type=True
    if data_type:

        for i in df.columns:
            df[i].dtypes==schema[i]
            logger.info(f"data column: {i} is matching with schema data set")
    else:
        logger.info("datatypes are not maching with schema data")        
        


In [60]:
validate_schema(df=df)

[2023-03-04 23:09:58,446: INFO: 2573481559]: data column: age is matching with schema data set
[2023-03-04 23:09:58,448: INFO: 2573481559]: data column: sex is matching with schema data set
[2023-03-04 23:09:58,449: INFO: 2573481559]: data column: bmi is matching with schema data set
[2023-03-04 23:09:58,451: INFO: 2573481559]: data column: children is matching with schema data set
[2023-03-04 23:09:58,453: INFO: 2573481559]: data column: smoker is matching with schema data set
[2023-03-04 23:09:58,457: INFO: 2573481559]: data column: region is matching with schema data set
[2023-03-04 23:09:58,458: INFO: 2573481559]: data column: expenses is matching with schema data set


In [62]:
from evidently.model_profile import Profile
from evidently.model_profile.sections import DataDriftProfileSection
from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import DataDriftTab
import json

c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\site-packages\evidently\analyzers\__init__.py:3: UserWarning: analyzers are deprecated, use metrics instead
  warnings.warn("analyzers are deprecated, use metrics instead")
c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\site-packages\evidently\model_profile\__init__.py:8: UserWarning: model profiles are deprecated, use metrics instead
  warnings.warn("model profiles are deprecated, use metrics instead")
c:\Users\somit\Downloads\project_ineuron\insurence_premium_prediction\env\lib\site-packages\evidently\dashboard\__init__.py:8: UserWarning: dashboards are deprecated, use metrics instead
  warnings.warn("dashboards are deprecated, use metrics instead")


In [64]:
con.

TypeError: 'ConfigurationManager' object is not callable

In [ ]:
def get_and_save_data_drift_report():
        try:
            profile =Profile(sections=[DataDriftProfileSection()])
            train_df=con().get_data_ingestion().train_file_path
            test_df= con().get_data_ingestion().train_file_path

            profile.calculate(train_df,test_df)
            report=json.loads(profile.json())
            report_file_path = self.data_validation_config.report_file_path
            report_dir = os.path.dirname(report_file_path)
            os.makedirs(report_dir,exist_ok=True)    

            write_json(report_file_path,report)
            return report
        except Exception as e:
            raise e    